In [1]:
!pip uninstall fitz PyMuPDF --yes
!pip install PyMuPDF
#!pip install PyMuPDF==1.19.1  ## install a release later than 1.19.0
#!pip install PyMuPDF==1.18.19  ## don't install 1.19.0 because of Wrong x0/y0 values for bboxes in get_text("words"), see: https://github.com/pymupdf/PyMuPDF/issues/1328
# ## Releases of PyMuPDF as of Oct 18, 2021: 1.11.2, 1.12.5, 1.13.20, 1.14.19.post2, 1.14.20, 1.14.21, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.16.7, 1.16.8, 1.16.9, 1.16.10, 1.16.11, 1.16.12, 1.16.13, 1.16.14, 1.16.15, 1.16.16, 1.16.17, 1.16.18, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.17.6, 1.17.7, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.18.6, 1.18.7, 1.18.8, 1.18.9, 1.18.10, 1.18.11, 1.18.12, 1.18.13, 1.18.14, 1.18.15, 1.18.16, 1.18.17, 1.18.18, 1.18.19, 1.19.0


Found existing installation: PyMuPDF 1.19.6
Uninstalling PyMuPDF-1.19.6:
  Successfully uninstalled PyMuPDF-1.19.6
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached PyMuPDF-1.19.6-cp39-cp39-macosx_10_9_x86_64.whl (7.6 MB)
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
!open -a Preview book-1.pdf book-2.pdf book-3.pdf
#just added it but what is the point of ... hmmm 

In [3]:
config = [
   {"filename" : "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf",
    "name" : "book-1",
    "page_count" : 642,
    "sections" : {
      "preface" : [1, 78],
      "toponymique" : [49, 76],
      "abbrev" : [77, 78],
      "content" : [79, 607],
      "sample" : [79, 79+10],
      "index" : [617, 639]},
    },
   {"filename" : "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf",
    "name" : "book-2",
    "page_count" : 725,
    "sections" : {
      "preface" : [1, 79],
      "abbrev" : [6, 7],
      "content" : [8, 700],
      "sample" : [8, 8+10],
      "index" : [704, 725]},
    },
   {"filename" : "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf",
    "name" : "book-3",
    "page_count" : 588,
    "sections" : {
      "preface" : [1, 7],
      "abbrev" : [6, 7],
      "content" : [8, 554],
      "sample" : [8, 8+10],
      "index" : [556, 583],
      "familyidx" : [584, 585]},
    },
]


In [4]:
import sys, fitz, re
from pprint import pprint

#SD: would be nice to make it semi automated:
"""SD
get features like font size use 
devide them into sub catagories with specific rangies with every font meaning a new thing: e.g bold -- can be a possible genus / largest are headers etc
find the headers and devide things based on headers 
Would be very nice to also find a way to find  the "indentation" of the words like if it's centered / left / right etc cause fitz doesn't pick that up and this can help with finding headers and stuff too.
"""

## Purpose: this cell is a pre-processing step that will create a symlink (e.g. book-1.pdf)
##          that points to the original file, and then will create new PDF documents
##          that are excerpts of subsets of the whole book
##          as named sections such as book-1-index.pdf or book-1-abbrev.pdf
##          The file book-X-sample.pdf contains the first 10 pages of the book-1-content.pdf
##          and the sample is used to speed up development/testing later on

## https://pymupdf.readthedocs.io/en/latest/document.html#Document.delete_pages
##    In general, the performance of this method is dependent on the number of remaining pages
##    NOT on the number of deleted pages
## So, keeping fewer pages is faster, and deleting more pages at once is also faster 
def excerpt(infile, outfile, start, end):
  doc = fitz.open(infile)  # open document
  last_page = doc.page_count
  print(start, end, last_page, outfile)
  deleteme = []
  if start - 1 > 0:
    deleteme.extend(list(range(0, start - 1)))       
  if end < last_page:
    deleteme.extend(list(range(end, last_page)))
  # print(deleteme)
  doc.delete_pages(deleteme)
  doc.save(outfile, garbage=4, clean=True)

import os, os.path
for book in config:
   symlink = book["name"] + ".pdf"
   fname = book["filename"]
   if os.path.exists(book["filename"]) and not os.path.exists(symlink):
      print("Creating symlink: ", symlink, " ->", fname)
      os.symlink(book["filename"], symlink)
   for section in book["sections"]:
      #print(s)
      start, end = book["sections"][section];
      out_fname = book["name"] + "-" + section + ".pdf"
      excerpt(book["filename"], out_fname, start, end)

!open -a Preview book-[1,2,3]-*.pdf

## Runtime of this cell is ~11m38s [Aaron]
## Runtime of this cell is ~187.9s (about 3 minutes!)[Shide]

1 78 642 book-1-preface.pdf
49 76 642 book-1-toponymique.pdf
77 78 642 book-1-abbrev.pdf
79 607 642 book-1-content.pdf
79 89 642 book-1-sample.pdf
617 639 642 book-1-index.pdf
1 79 725 book-2-preface.pdf
6 7 725 book-2-abbrev.pdf
8 700 725 book-2-content.pdf
8 18 725 book-2-sample.pdf
704 725 725 book-2-index.pdf
1 7 588 book-3-preface.pdf
6 7 588 book-3-abbrev.pdf
8 554 588 book-3-content.pdf
8 18 588 book-3-sample.pdf
556 583 588 book-3-index.pdf
584 585 588 book-3-familyidx.pdf


In [15]:
# !open -a Preview book-[1,2,3]-*.pdf


In [11]:
!open -a Preview book-[1,2,3]-index.pdf
#SD: First one is corrupted? doesn't open

In [7]:
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics

import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)
print(Face("Times", 10.929, 0, 4))



Font: 'Times' (serifed, proportional), size 10.929, color #000000


'italic, serifed, proportional'

In [13]:
import sys, fitz, re
from pprint import pprint
# fname = "book-1-sample.pdf"
fname = "book-1.pdf"
doc = fitz.open(fname)  # open document

from fitz.utils import getColor
#blue = getColor("aliceblue")

# txtpgoffset = 77
txtpgoffset = 0
# start_page = 1 + txtpgoffset
start_page = 1
#end_page = start_page + 11
#end_page = start_page + 150
#end_page = start_page + 530
end_page = doc.page_count
items = []
mode = "searching"
print(mode)
for i in range(start_page, end_page):
    page = doc[i]
    # read page text as a dictionary, suppressing extra spaces in CJK fonts
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", flags=flags)["blocks"]
    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        #annot = page.add_rect_annot(br)
        for y,l in enumerate(b["lines"]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            for z,s in enumerate(l["spans"]):  # iterate through the text spans
                if z==0 and mode == "found":
                    mode = "spanning"
                #s["style"] = flags_decomposer(s["flags"])  # readable font flags
                face = Face(s["font"], s["size"], s["color"], s["flags"])
                t = s["text"]
                ## FIXME: [83, 'N  Cystopteris  filix-fragilis  (L.) Borb. —  Polypodiumfilix-fragile  L.,  Cyst,  fragilis']
                ## FIXMECLASS: OCR Error; a spurious mark in the text preceding the genus is interpreted as a letter 'N'
                if z==0 and mode != "found" and re.search(r"Times-Bold\b", str(face)) and s["size"] > 10.0:
                        r_rect = fitz.Rect(lr)
                        r_high = fitz.Rect(s["bbox"])
                        joined = " ".join(list(map(lambda x: x["text"].strip(), l["spans"])))  ## FIXMEDESC: ideally this simplistic code would work if not for OCR Errors
                        item = joined
                        # found = re.match(r"(?P<genus>([A-Z]\w+) (?P<species>\w+)", joined)
                        # d = found.groupdict()
                        matches = re.match(r"([A-Z]\w+) (\w+)", joined)
                        warning = "Possible OCR Error"
                        if matches and len(matches.groups()) == 2:
                            found = matches.groups()
                            warning = "Normal"
                            mode = "found"
                            # item = f"""${d['genus']} ${d['species']}"""
                            item = " ".join(found)
                        else:
                            warning = "Possible OCR Error"

                        print([i, x, y, z, item, warning])
                        annot_high = page.add_highlight_annot(r_high)
                        items.append([t, r_rect, r_high, i, z,s, y,l, x,b, warning])
print(len(items))
#print(i, items[0][:3])


searching
[34, 0, 0, 0, 'r', 'Possible OCR Error']
[59, 9, 0, 0, 'J', 'Possible OCR Error']
[77, 8, 5, 0, 'JL', 'Possible OCR Error']
[78, 8, 0, 0, 'Lycopodium cernuum', 'Normal']
[79, 1, 0, 0, 'Selaginella denticulata', 'Normal']
[79, 16, 0, 0, 'Isoetes hystrix', 'Normal']
[79, 24, 1, 0, 'Isoetes olympica', 'Normal']
[80, 13, 0, 0, 'Equisetum maximum', 'Normal']
[80, 18, 0, 0, 'Equisetum palustre', 'Normal']
[80, 25, 0, 0, 'Equisetum ramosissimum', 'Normal']
[81, 9, 0, 0, 'Ophioglossum vulgatum', 'Normal']
[81, 11, 3, 0, 'Ophioglossum lusitanicum', 'Normal']
[81, 21, 0, 0, 'Osmunda regalis', 'Normal']
[82, 9, 0, 0, 'Gymnogramma leptophylla', 'Normal']
[82, 17, 0, 0, 'Cheilanthes pteridioides', 'Normal']
[83, 28, 0, 0, 'Âdiantum capillus-veneris I (PL III, n. 4). — 2|. Rhizome rampant à écailles', 'Possible OCR Error']
[84, 10, 0, 0, 'Pteris longifolia', 'Normal']
[84, 16, 0, 0, 'Pteridium aquilmum', 'Normal']
[85, 1, 0, 0, 'Athyrium filix', 'Normal']
[85, 12, 0, 0, 'Dryopterîs aculeat

In [14]:
## This cell requires ~20 mins to calculate results from the first book

import math
doc = fitz.open(fname)  # open document
#subset = items[:6]
subset = items
boxes = []
for it, item in enumerate(subset):
    mode = "searching"
    if (it >= len(subset) - 1):
        break
    next_item = subset[it + 1]
    [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0] = item
    [t, rr, rh, i, z,s, y,l, x,b, warning] = [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0]
    [t1, rr1, rh1, i1, z1,s1, y1,l1, x1,b1, warning1] = next_item
    page = doc[i0]
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", flags=flags)["blocks"]
    blocks = blocks[x:]
    r_rect = rr0
    
    #print(t0)
    # if t0 == "Isoetes olympica":
    #print([int(it), len(subset), rr0, t0, "\n", t1])
    
    annot_rect = page.add_rect_annot(rr0)
    annot_rect.set_border(width=1, dashes=[1,2])
    annot_rect.update()

    
    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        #annot = page.add_rect_annot(br)
        for y,l in enumerate(b["lines"][y0:]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            s = l["spans"][0]
            #for z,s in enumerate(l["spans"]):  # iterate through the text spans
            this_line = " ".join(list(map(lambda x: x["text"], l["spans"])))
            if mode != "found":
                if re.search("Aire g", this_line):
                    mode = s["font"] + " " + str(s["size"])
                    r_rect = r_rect.include_rect(lr)
                    #print(str((x, y, s["size"], s["font"])) + "FOUND: " + this_line)
                elif lr.intersects(rr1):                    
                    mode = "found"
                    #print(str((x, y, s["size"], s["font"])) + "FOUND: " + this_line)
                else:
                    if mode != "searching":
                        (font, size) = mode.split(" ")
                        isize = round(math.ceil(float(size)))
                        if s["size"] >= float(isize):
                            mode = "found"
                    if mode != "found":
                        r_rect = r_rect.include_rect(lr)
                        # if y >= y0:
                        #     r_rect = r_rect.include_rect(lr)
                    
    r_high = rh0
    annot_rect = page.add_rect_annot(r_rect)
    annot_high = page.add_highlight_annot(r_high)
    # boxes.append([t0, start_page + it, r_rect, item]) . ## FIXME: thinking it should be i0 not start_page + it
    boxes.append([t0, i0, r_rect, item])


# print(boxes[0])
print(len(boxes))

#doc.delete_pages(end_page, doc.page_count - 1)
#doc.delete_pages(0, start_page - 1)
marked_epithet_fname = "marked-pages-" + doc.name
doc.save(marked_epithet_fname, garbage=4, clean=True)
#!open -a Preview marked-pages-book-1-sample.pdf
!open -a Preview marked-pages-book-1.pdf



1064


In [15]:
import pprint
pp = pprint.PrettyPrinter(compact=True)

import math
doc = fitz.open(fname)  # open document
results = {}
for it, thing in enumerate(boxes):
    mode = "searching"
    #print(thing)
    [name, pageno, rrr, item] = thing
    page = doc[pageno]
    [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0] = item
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", clip=rr0, flags=flags)["blocks"]    


    print(pageno - txtpgoffset, i0, t0)
    results[t0] = {}
    results[t0]["name"] = t0
    results[t0]["name_rects"] = [rr0]
    results[t0]["pdf_page"] = pageno
    results[t0]["book_page"] = pageno - txtpgoffset
    key = "Extra"
    mode = "searching"
    margins = [[(math.ceil(l["bbox"][0]), l["spans"][0]["text"].split(" ")[0]) for l in filter(lambda bl: bl.get("spans"), b["lines"])] for b in blocks]
    margins = [m for n in margins for m in n]
    targets = list(map(lambda n: n[0], filter(lambda m: m[1] in ["L.", "S.", "Aire"] , margins))) or [blocks[0]["lines"][0]["bbox"][0]]
    margins_max = max(targets or margins)
    margins_min = min(list(map(lambda n: n[0], margins)))

    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        for y,l in enumerate(b["lines"][y0:]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            xloc = min(margins_max, math.ceil(lr.x0))
            this_line = re.sub(r"\s+", " ", " ".join(list(map(lambda x: x["text"].strip(), l["spans"])))).strip()
            # matches = re.match("^(?P<label>L\.|S\.|Aire g\w+\.|Not\w+\.|Fleurs|Fructif\.|Fructific|Floraison\w+)\s", this_line)
            #### FIXME: Phyllitis scolopendrium (L.) Newm. —Asplenium scolopendrium L., Scolopendrium
            #### FIXME: Juniper
            matches = re.match("^(?P<label>" + t0 + "|.*?)(\s|$)", this_line)
            if matches:
                d = matches.groupdict()
                if margins_max - xloc < 5:
                    key = d["label"]
                    if ' ' in key:
                        key = "Description"
                    elif key not in ["Description", "L.", "S.", "Aire"]:
                        key = "Extra"
                    elif key in ["Description", "L.", "S.", "Aire"]:
                        pass
                    else:
                        print("UNHANDLED: " + key)
                        key = "Extra"
                frag = results[t0].get(key) or ""
                results[t0][key] = " ".join([frag.strip(), this_line.strip()]).strip()
                key_rects = key + "_rects"
                if not results[t0].get(key_rects):
                    results[t0][key_rects] = []    
                results[t0][key_rects].append(lr)
                #print([x, y, xloc, mode, this_line])
            else:
                print(["ERROR:", x, y, xloc, mode, this_line])
    # pp.pprint(results[t0])
    print()
    #break
                    
    # r_high = rh0
    # annot_rect = page.add_rect_annot(r_rect)
    # annot_high = page.add_highlight_annot(r_high)
    # boxes.append([t0, r_rect, item])




34 34 r

59 59 J

77 77 JL

78 78 Lycopodium cernuum

79 79 Selaginella denticulata

79 79 Isoetes hystrix

79 79 Isoetes olympica

80 80 Equisetum maximum

80 80 Equisetum palustre

80 80 Equisetum ramosissimum

81 81 Ophioglossum vulgatum

81 81 Ophioglossum lusitanicum

81 81 Osmunda regalis

82 82 Gymnogramma leptophylla

82 82 Cheilanthes

83 83 Âdiantum capillus-veneris

84 84 Pteris longifolia

84 84 Pteridium aquilmum

85 85 Athyrium

85 85 Dryopterîs aculeata

85 85 Dryopteris

86 86 Blechnum

87 87 Ceterack officinarum

88 88 Asplenium bourgaei

88 88 Asplenium

88 88 Asplenium ruta-muraria

88 88 Asplenium

88 88 Asplenium

89 89 Phyllitis scolopendrium

89 89 Phyllitis hemionitis

90 90 Polypodium vulgare L.

90 90 Salvinia natans

92 92 Abies cilicica

92 92 Cedrus libani

93 93 Pinus pinea

93 93 Pina

93 93 Pinus halepensis

94 94 Cupressus sempervirens

95 95 Juniperus oxycedrus

96 96 Juniper

96 96 Juniperus

96 96 Arceuthos drupacea

97 97 Ephedra campylopoda

97 97 

In [16]:
from tqdm import tqdm
import math
# marked_fname = "marked-pages-book-1-sample.pdf"
marked_fname = "marked-pages-book-1.pdf"
#doc = fitz.open(fname)  # open document
doc = fitz.open(marked_fname)  # open document
print(len(results))

from fitz.utils import getColorList
cl = getColorList()
from fitz.utils import getColor

pink = getColor("lightpink")
green = getColor("aquamarine")
blue = getColor("lightskyblue")
gray = getColor("whitesmoke")
yellowish = getColor("antiquewhite")
fills= [pink, green, blue, gray, yellowish]

fc = {"L.":pink, "S.":blue, "Aire":green, "Description":gray, "Extra":yellowish}
for it, name in enumerate(tqdm(results.keys())):
    item = results[name]
    #print(item)
    #print(item["Description"])
    book_page = item["book_page"]
    pdf_page = item["pdf_page"]
    print([it, name, book_page, pdf_page])
    page = doc[book_page - 1]
    for k in filter(lambda x: not re.search(r"_page|_rect", x), item.keys()):
        rects = item[k + "_rects"]
        for r in rects:
            #print(k, r)
            
            if not page.rect.intersects(r):
                print(["ERROR: ", k, r])
            else:
                # #annot = page.add_highlight_annot(fitz.Rect(r))
                # annot_rect = page.add_rect_annot(fitz.Rect(r))
                # #annot_rect.set_colors(stroke=(0,1,0))
                # annot_rect.set_border(width=1, dashes=[1,2])
                # annot_rect.update()

                if k in ["L.", "S.", "Aire", "Description", "Extra"]:
                    print(["HIGHL: ", fc[k], k, r])
                    annot = page.add_highlight_annot(fitz.Rect(r))
                    annot.set_colors(stroke=fc[k])
                    annot.update()
                    #annot_rect = page.add_rect_annot(fitz.Rect(r))
                    #annot_rect.set_colors(stroke=green)
                    #annot_rect.set_border() #(width=1, dashes=[1,2])
                    #annot_rect.update()
                    #annot_rect.update(fill_color=red)


# doc.delete_pages(end_page, doc.page_count - 1)
# doc.delete_pages(0, start_page - 1)
doc_fname = "results-" + doc.name
doc.save(doc_fname, garbage=4, clean=True)
print(doc_fname)
# !open -a Preview results-marked-pages-book-1-sample.pdf
!open -a Preview results-marked-pages-book-1.pdf



862


  1%|          | 6/862 [00:00<00:15, 53.94it/s]

[0, 'r', 34, 34]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(19.200000762939453, -8.104736328125, 30.18215560913086, 16.888263702392578)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(259.9200134277344, 51.3499755859375, 346.751708984375, 62.3499755859375)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(515.7293090820312, 51.3499755859375, 555.86376953125, 62.3499755859375)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(76.08000183105469, 80.780029296875, 556.5726318359375, 94.780029296875)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(48.720001220703125, 96.1400146484375, 256.5834045410156, 110.1400146484375)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(76.08000183105469, 116.05999755859375, 556.6231689453125, 130.05999755859375)]
['

  3%|▎         | 25/862 [00:00<00:06, 130.00it/s]


['HIGHL: ', (0.4980392156862745, 1.0, 0.8313725490196079), 'Aire', Rect(36.0, 404.28997802734375, 79.35308837890625, 413.28997802734375)]
[23, 'Asplenium bourgaei', 88, 88]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(42.0, 54.6300048828125, 408.07354736328125, 65.6300048828125)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(20.639999389648438, 66.1500244140625, 404.6177978515625, 77.1500244140625)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(20.399999618530273, 77.66998291015625, 404.9436950683594, 88.66998291015625)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(20.399999618530273, 89.19000244140625, 405.2630310058594, 100.19000244140625)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(20.15999984741211, 100.71002197265625, 404.9595031738281, 11

...


['HIGHL: ', (1.0, 0.7137254901960784, 0.7568627450980392), 'L.', Rect(57.599998474121094, 572.1400146484375, 400.783203125, 582.1400146484375)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(57.84000015258789, 585.0999755859375, 214.06600952148438, 595.0999755859375)]
['HIGHL: ', (0.4980392156862745, 1.0, 0.8313725490196079), 'Aire', Rect(57.36000061035156, 598.0599975585938, 276.94500732421875, 608.0599975585938)]
[823, 'Silène caryophylloides', 576, 576]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(25.200000762939453, 402.8699951171875, 409.9945068359375, 413.8699951171875)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(24.959999084472656, 414.6300048828125, 227.8140106201172, 425.6300048828125)]
[824, 'Silène amana', 579, 579]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(62.63999938964844, 52.30999755859375, 4

 98%|█████████▊| 843/862 [00:09<00:00, 121.50it/s]


['HIGHL: ', (1.0, 0.7137254901960784, 0.7568627450980392), 'L.', Rect(34.560001373291016, 349.9800109863281, 401.29461669921875, 359.9800109863281)]
['HIGHL: ', (1.0, 0.7137254901960784, 0.7568627450980392), 'L.', Rect(16.559999465942383, 359.5799865722656, 398.269287109375, 379.17999267578125)]
['HIGHL: ', (0.5294117647058824, 0.807843137254902, 0.9803921568627451), 'S.', Rect(34.79999923706055, 380.94000244140625, 237.82150268554688, 390.94000244140625)]
['HIGHL: ', (0.4980392156862745, 1.0, 0.8313725490196079), 'Aire', Rect(34.31999969482422, 392.70001220703125, 141.10240173339844, 402.70001220703125)]
[843, 'Gypsophila polygonoides', 592, 592]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(38.15999984741211, 409.22998046875, 401.8318176269531, 420.22998046875)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(17.280000686645508, 420.989990234375, 401.82550048828125, 431.989990234375)]
['HIG

100%|██████████| 862/862 [00:09<00:00, 92.39it/s] 


['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(35.52000045776367, 279.9700012207031, 421.0865783691406, 288.9700012207031)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(35.52000045776367, 289.57000732421875, 420.41497802734375, 298.57000732421875)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(35.279998779296875, 299.1700134277344, 255.68563842773438, 308.1700134277344)]
['HIGHL: ', (0.5294117647058824, 0.807843137254902, 0.9803921568627451), 'S.', Rect(54.0, 237.25, 63.675270080566406, 246.25)]
['HIGHL: ', (0.4980392156862745, 1.0, 0.8313725490196079), 'Aire', Rect(53.52000045776367, 258.6099853515625, 260.0776672363281, 267.6099853515625)]
[849, 'Kohlrauschia velutina', 595, 595]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(56.880001068115234, 356.4700012207031, 420.54376220703125, 367.4700012207031)]
['HIGHL: 

In [17]:
import pandas as pd

df = pd.DataFrame.from_dict(results.values()).fillna('')
df = df[df.columns.drop(list(df.filter(regex='_rects')))]
df.to_csv("book-1.csv", index=False)
df

,name,pdf_page,book_page,Extra,Description,L.,Aire,S.
0,r,34,34,r INTRODUCTION XXXVII Occupant des aires très ...,,,,
1,J,59,59,J Lettre alternant dans la transcription de mo...,,,,
2,JL,77,77,Nahal Nik Np Ol P Pb Pr Reese Russ Sam Schw Th...,,,,
3,Lycopodium cernuum,78,78,Fructification de novembre à mars. Sur grès tr...,Lycopodium cernuum L. var. capillaceum Willd (...,"L. Mi. Entre Nahr es Safa et 'Aïn Qa'a, 1942 e...",Aire géogr. —- Régions tropicales et subtropic...,
4,Selaginella denticulata,79,79,Végétation active de novembre à avril. Fructif...,Selaginella denticulata (L.) Link — Lycopodium...,"L. Ct. et ML, Ce. Saïda (Bl), Beyrouth et envi...",Aire géogr. —• Tour de la Méditerranée. Madère...,"S. Non signalée. Présence presque certaine, Ct..."
...,...,...,...,...,...,...,...,...
857,Dianthus strictus,599,599,Floraison: mai-décembre. CC. tous terrains. Le...,Dianthus strictus Banks et Sol. (non Sibth. et...,,,
858,Dianthus judaicus,603,603,Var. auraniticus (Post.) n. comb. — Calice 3 c...,Dianthus judaicus Boiss. — D. pattens Sibth. e...,L. Sy. Baalbeck (Wall). St. Qamou'at Hermel (P...,"Aire géogr. — Turquie sud, Syrie, Liban, Pales...","S. A.L. Ouadi-el-Qarn (Sam, Mt, Pb), Zebdani, ..."
859,Dianthus pachypetalus,603,603,,Dianthus pachypetalus Stapf— D. floribundus Bo...,,,
860,Dianthus crinitus,604,604,Floraison: mai-juin.,"Dianthus crinitus Smith (PI. CLXXXV, n. 4). — ...",,"Aire géogr. — Turquie, Géorgie, Iran, Bélouchi...",S. St. Zélaf (Pb).
